# Create a *RAG* for the Azure Preparation information in a .txt

In [1]:
# import the libraries 

import os # load the .env file 
from dotenv import load_dotenv # load the .env file

from langchain_community.document_loaders import TextLoader # load the text and convert into langchain object
from langchain_text_splitters import RecursiveCharacterTextSplitter # split the whole document into chunks 


from langchain_chroma import Chroma # provide the vector database 
from langchain_huggingface.embeddings import HuggingFaceEmbeddings # for the hugging face embeddings

from langchain.messages import SystemMessage, HumanMessage # for the messages format 
from langchain_openai import ChatOpenAI # for the openai format 

from IPython.display import display, Markdown
import gradio as gr

/home/enyo710/projects_ubuntu/cv_projects/azure_prep/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the .env file 
load_dotenv(override=True)

True

In [3]:
# provide the loader of the file  
loader = TextLoader(file_path='azure_prep/Azure.txt', encoding='utf-8')

# load the file 
file = loader.load()
file

[Document(metadata={'source': 'azure_prep/Azure.txt'}, page_content='Core services like computing ,networking, storage, and databases.\n\nAzure fundamentals:Cloud architecture and azure\n==Study 1 to 2 hours a day==\n\nIn Person Vue (Proctored exam system)\n700/1000 around 70% to pass (afford to get wrong 10 - 14 questions)\n\nSkills measured \nCloud concepts (25 - 30%)\n    -13 to 15 quesions\n\nAzure arquitecture and services (35 - 40%)\n    -16 to 18 questions\n\nAzure management and governance (30 - 35%)\n    -14 to 16 questions\n\n35 to 50 questions\n\nQuestions format \n-Multiple choice\n-Multiple answer\n-Drag and drop\n-Yes and No\n\n|===Exam AZ-900===|\n\n1.Cloud concepts\n===What is cloud computing?===\nUsing a network in a remote server that is hosted on the internet, so that you can store, \nmanage, and procees data ,rather than a local server or PC (someone else does the server job)\n-someone owns the server\n-someone hires the IT people\n-someone pays or rents real-estate

In [4]:
# display the content of the file 
Number_of_characters = 2000 # number of characters to truncate 

display(Markdown(file[0].page_content[:Number_of_characters])) # truncate to "N" characters

Core services like computing ,networking, storage, and databases.

Azure fundamentals:Cloud architecture and azure
==Study 1 to 2 hours a day==

In Person Vue (Proctored exam system)
700/1000 around 70% to pass (afford to get wrong 10 - 14 questions)

Skills measured 
Cloud concepts (25 - 30%)
    -13 to 15 quesions

Azure arquitecture and services (35 - 40%)
    -16 to 18 questions

Azure management and governance (30 - 35%)
    -14 to 16 questions

35 to 50 questions

Questions format 
-Multiple choice
-Multiple answer
-Drag and drop
-Yes and No

|===Exam AZ-900===|

1.Cloud concepts
===What is cloud computing?===
Using a network in a remote server that is hosted on the internet, so that you can store, 
manage, and procees data ,rather than a local server or PC (someone else does the server job)
-someone owns the server
-someone hires the IT people
-someone pays or rents real-estate
-You configure your cloud services and code, someone else does the rest

=Evolution of cloud hosting=
1995 - Dedicated server
One Physical machine dedicated to single a business
-runs a single web-app/site
(Very expensive, high maintenance, *High security)

Virtual Private server (VPS)
One Physical machine dedicated to a single business (virtualized into submachines)
-Runs multiple web-apps/sites
(Better utilization and Isolation of resources)

Shared Hosting (GoDaddy)
One Physical machine, shared by hundred of business 
-Relies on most tenants under-utlizing their resources
(Very Cheap, limited functionality , Poor isolation)

Cloud hosting
Multiple physical machines (act as one system)
-Abstracted into multiple cloud services
(Flexible, Scalable, Secure, Cost-effective, High configurability)


==Common cloud Services==
A cloud provider can have hundreds of cloud services 
(grouped into types of services)

=Four most common types of cloud services for InfrastructureAsAService(IaaS)=

-Compute
Virtual Computer that can run application, programs and code

-Storage
Virtual hard-drive tha

In [5]:
# split the characters into chunks 

# instantiate a splitter object
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# split the documents into chunks 
chunks = splitter.split_documents(documents=file)


In [6]:
# see how many chunks were created
print(len(chunks))

68


In [7]:
# take a quick peek at the chunks
chunks

[Document(metadata={'source': 'azure_prep/Azure.txt'}, page_content='Core services like computing ,networking, storage, and databases.\n\nAzure fundamentals:Cloud architecture and azure\n==Study 1 to 2 hours a day==\n\nIn Person Vue (Proctored exam system)\n700/1000 around 70% to pass (afford to get wrong 10 - 14 questions)\n\nSkills measured \nCloud concepts (25 - 30%)\n    -13 to 15 quesions\n\nAzure arquitecture and services (35 - 40%)\n    -16 to 18 questions\n\nAzure management and governance (30 - 35%)\n    -14 to 16 questions\n\n35 to 50 questions\n\nQuestions format \n-Multiple choice\n-Multiple answer\n-Drag and drop\n-Yes and No\n\n|===Exam AZ-900===|\n\n1.Cloud concepts\n===What is cloud computing?===\nUsing a network in a remote server that is hosted on the internet, so that you can store, \nmanage, and procees data ,rather than a local server or PC (someone else does the server job)\n-someone owns the server\n-someone hires the IT people\n-someone pays or rents real-estate

In [3]:
# instantiate a name for the vector database 
vector_db_name = 'azure_prep'

# name of the embedding to use from hugging face 
all_mini = 'all-MiniLM-L6-v2'

# provide the collection name 
collection_name = 'collection_azure_prep'

In [5]:
# provide the embedding model to use in the vector database 
embedding_model = HuggingFaceEmbeddings( # use endpoint embeddings to load a Hugging face sentence-transfomers's model
    model=all_mini
)

In [10]:
embedding_model

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
# provide the vector database 
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding_model,
    persist_directory=vector_db_name,
    collection_name=collection_name
)


## Load the vector database already created 

In [6]:
# instantiate the chroma database directory object (load the vector database)
load_vector_db = Chroma(
    persist_directory=vector_db_name,
    embedding_function=embedding_model,
    collection_name=collection_name
)

In [7]:
# try the retriever to see how accurate it is 
retriever = load_vector_db.as_retriever(search_type="similarity", search_kwargs={"k":10})


In [8]:
answer = retriever.invoke("Provide the Storage Services that Azure has")
content_answer = "".join([content.page_content for content in answer])

In [16]:
display(Markdown(content_answer))
#len(content_answer)

=Azure Services that work with Private link:
        -Azure storage
        -CosmoDB
        -SQL

    =PRIVATE LINK SERVICE allows to connect own workload to Private Link.
    (Need AZURE STANDARD INTERNAL LOAD BALANCER and associate it with the link service)

8.Storage Accounts

==Storage Services==
    
    =AZURE BLOB SERVICE 
        - (OBJECT SERVERLESS STORAGE) Store large files and large amounts of unstructured files 
        Pay for what you store, unlimited storage, no-resizing volumens, filesystem protocols.


    =AZURE DISK STORAGE
        -Virtual volume, choose SSD or HDD, encryption by default, attach volume to VMs


    =AZURE FILE STORAGE 
        -shared volume that you can access and manage like a file server (SMB -> Server Message Block protocol)


    =*AZURE QUEUE STORAGE 
        -(MESSAGING QUEUE) A data store for queuing and reliably delivering messages between applications


    =*AZURE TABLE STORAGE 
        -(WIDE-COLUMN NOSQL DATABASE) A NoSql store that hosts unstructured data 
        independent of any schema
               
    
    =AZURE DATA BOX / AZURE DATABOS HEAVY
        -A rugged briefcase computer and storage designed to move 
        TERABYTES OR PETABYTES of data

    =AZURE ARCHIVE STORAGE 
        -Long term cold storage for when you need to hold onto files 
        for years on the cheapest storage options

    =AZURE DATA LAKE STORAGE 
        -A centralized repository that allows you to store 
        structured and unstructured data at any scale


==Introduction to Storage Accounts==
    -Contains all of Azure storage data objects such as:
        -Blobs
        -files
        -queues
        -tables
        -disks
          
    Several types of storage accounts(DIFFERENT FEATURES , OWN PRICING MODELS):

        (Storage Type = Account kind)

        -General purpose v1 (legacy)
        -General purpose v2 
        -Blob storage (legacy)
        -BLockBlobStorage
        -FileStorage=Four most common types of cloud services for InfrastructureAsAService(IaaS)=

-Compute
Virtual Computer that can run application, programs and code

-Storage
Virtual hard-drive that can store files

-Networking
Virtual network being able to define internet connections or Network isolations

-Databases
Virtual database for storing reporting data or a database for general purpose web-application


==What is Microsoft?==
American multinational computer technology corporation (headquartered in Redmond, Washington)
-Makes sofware , phones, tablets, game consoles, CLOUD SERVICES, search engine
-Around since late 1970s (well known for operating system Windows)

==What is Azure==
Cloud provider service for Microsoft (CSPs = Cloud Service Provider)
Azure means "Bright blue color of the cloudless sky"

==Benefits of Cloud Computing==
-Cost effective
    You pay what you consume (no up-front cost), Pay-as-you-go(PAYG)
    thousands of customers sharing the cost of the resources

-Global
    Launch workloads anywhere in the world, just choose region

-Secure
    Cloud provider takes care of physical security.
    Can be secure by default or you can configure access down to granular level

-Reliable 
    Data backup, disaster recovery, data replication, fault tolerance

-Scalable 
    Increase or decrease recources and services based on demand

-Elastic
    Automate scaling during spikes and drop in demand

-Current 
    Hardware and managed sofware is patched ,upgraded and replaced by cloud provider 
    (without interruptions, or few interruptions)


==Types of cloud computing==
Software As A Service (SaaS)
    - Product that is run and managed by the service Provider
    (Don't need to worry about how the service is maintained, just remains available)
    -SalesForce, Gmail, Office 365-APPEND BLOBS:
            -Optimized for append Operations
            -Ideal for scenarios such as LOGGING DATA from Virtual Machine


        -PAGE BLOBS:
            -Store random access files up to 8 TB in size 
            -Store virtual hard drive (VHD) files and serve as disks for Azure Virtual Machine

=AZ Copy=
    -COMMAND UTILITY that you can use to copy BLOBS OR FILES to or from a storage account

    1.Executable File you download 
    
    2.Need to have level of authorization via attach roles 
        -To download:
            Storage Blob Data Reader 

        -To upload:
            Storage Blob data Contributor
            Storage Blob data Owner 

    3. Gain access either via:
        -Azure Active Directory (AD)
        -Shared Access signature (SAS)

    4.Use COPY COMMAND to UPLOAD and DOWNLOAD
        

==Azure Storage Explorer==
    -STANDALONE APP that makes it easy to work with Azure Storage data on WINDOWS, macOS, and LINUX.
    (Create Blob Containers, upload files, create snapshots of disk, and more)

    

==Introduction to Azure files==
    -Fully managed FILE SHARE in the cloud
    
    FILE SHARE - CENTRALIZED SERVER FOR STORAGE that allows MULTIPLE CONNECTIONS 
    (Having one big shared drive that everyone (Virtual Machines) can work on at the same time)
    
    To Connect to the file share a NETWORK PROTOCOL you use:
        -Server Message Block (SMB)
        -Network File System (NFS)

    When the connection is established the "file share's filesystem 
    will be accessible in the specific directory within your own directory tree" (known as MOUNTING)--------------------------------------------------------------------------------
        =In Disk Section

        =AZURE DISKS=
            -BLOCK-LEVEL STORAGE volumes for Azure VMs


==Performance Tiers (Blob storage)==

    IOPS(INPUT/OUTPUT PER SECOND)->The higher the IOPS the faster the drive can read and write 

    2 types of performance:
        -Standard 
            -Stored in HDDs (Has moving parts, arm that needs to read and write data sequential to a disk
            ,good for writing and reading large amounts of data that is close together)
            -Varied performance based on access tier (Hot, cool, archive)
            
            -Use cases:
                -Backup and disaster recovery 
                -Media content 
                -Bulk data processing 


        -Premium
            -Stored on SSDs (SSD have no moving parts and data is distributed randomly -> That is why it reads and writes so fast )
            -Optimize low latency 
            -Higher throughput 

            -Use cases:
                -Interactive workloads 
                -Analytics 
                -AI or ML 
                -Data transformation 


==Access Tiers==
    - 3 types of access tiers for STANDARD STORAGE
        (Blob storage)
        -Hot 
            -Data accessed frequently
            -Highest storage cost, lowest access cost 
            
            Use case:
                Active use or expected to be accessed frequently
                Data staged for proccesing and eventual migration to the cool access tier

        -Cool
            -Data that's infrequently accessed and stored for at least 30 Days 
            -Lower storage cost, higher access costWhen the connection is established the "file share's filesystem 
    will be accessible in the specific directory within your own directory tree" (known as MOUNTING) 


==Azure FIles Use Cases==
    -Completely REPLACE or SUPPLEMENT  On-premsises file servers Network Attach Storage (NAS) devices
    
    -LIFT AND SHIFT your On-premise storage to the cloud via Classic Lift or Hybrid Lift
        Lift and shift: move workloads without rearchitecting (importing local VMs to the cloud)
        
        -Classic Lift:
            Where both the application and its data are moved to Azure 

        -Hybrid lift:
            Where the application data is moved to Azure files (APPLICATION continues to run on-premises)

    -Simplify cloud development 
        -Shared application settings:
            Multiple VMs and developer workstations need to access the same config files

        -Diagnostic Share:
            All VMs log to the file share, developers can mount and debug all logs in a centralized place 

        -Dev/Test/Debug:
            Quickly share tools for developer needed for local environments 

    -Containerization 
        Use Azure Files to persist volumes for stateful containers 


    Why use AZURE FILES instead of setting own FILE SHARE SERVER?
        -Shared Access:
            Alrady setup to work with standard networking protocols SMB and NFS 

        -Fully managed:
            Its kept up to date with security patches (designed to scale)

        -Scripting and Tooling:
            Automate the management and creation of file shared with AZURE API and POWERSHELL 

        -Resiliency:
            Build to be durable and always working 

==Azure File Sync==
    Service that allows you to CACHE Azure FIles share on an 
    ON PREMISES WINDOWS SERVER or CLOUD VMOperating systems implement a command-line inteface 
            in a shell or terminal 

    -The AZURE CLI can be installed on Windows, Mac and Linux.
    (Once installed you can type 'az' followed by other commands to 
    CREATE, UPDATE, DELETE, VIEW and MANAGE Azure resources)

==Visual Studio Code==
    -FREE SOURCE CODE EDITOR made by Microsoft for Windows, Linux and macOS


==Introduction to Azure Resource Manager(ARM)==   
    -Service that allows you to MANAGE Azure resources 

    -Collection of services in the Azure Portal, SO YOU CAN'T SIMPLY TYPE
    in "Azure Resource Manager"

    =Management layer that allows to:
        -Create, Update, Delete resources 
        -Apply management features (Access controls, Locks, tags)
        -Writing infrastructure as Code (IaC) via JSON templates 

        =Features that make up the ARM LAYER=
            -Subscriptions 
            -Management Groups 
            -Resource Groups 
            -Resource Providers 
            -Resource Locks 
            -Azure Blueprints 
            -Resource Tags 
            -Access Controls (IAM)
            -Role Based Acess Control (RBAC)
            -Azure policies 
            -ARM templates 

==ARM (use cases)==
    -Think of ARM as a GATEKEEPER 
        (All REQUESTS flow through ARM and it decides whether that 
        request can be performed on a RESOURCE)


==Azure Resource Manager(scope)==   
    =Scope=
        BOUNDARY OF CONTROL for Azure resources.
        Way to GOVERN your resource by placing resources
            -Withing a local grouping 
            -Applying logical restrictions in the form of rules 

    =Scopes=
        -Management Groups
            Logical grouping of multiple subscriptions 

            -Subscriptions 
                Grants access to Azure services based on billing and 
                support agreement 

                -Resource group
                    Logical grouping of multiple resources-Keep application and user data secure
        (applying right access controls to users and devices with 
        AZURE ACTIVE DIRECTORY CONDITIONAL ACCESS)


5. Azure App Service

==Azure App service==
    -Quickly DEPLOY AND MANAGE WEB APPS on Azure (without worrying of infrastrucutre)
    (Known as PaaS(Platform as a Service))

     HTTP BASED-SERVICE for hosting web applications, Rest APIs, and mobile backends
     (It is the Heroku of Azure)

     =Azure app service takes care of=
        -Security Patches for OS and languages
        -Load balancing
        -Autoscaling
        -Automated manager

    = Integrations and features =
        -Azure DevOps (DEPLOYMENTS)
        -Github Integration
        -Docker Hub integration 
        -Package management 
        -Easy to setup starting environments 
        -Custom domains
        -Attaching TLS/SSL certificates

    =Pay based on Azure app service plan=
        -Shared Tier:
            Free, shared (linux not supported)

        -Dedicated Tier:
            Basic, standard , premium, premiumV2 , premiumV3

        -Isolated Tier:
            
            (Azure App Services can also run dockers single mutli containers)


==Runtimes==

Runtime Environments 
    -Runtime software/Instructions that are executed 
    WHILE your program is running 

Runtime (Generally)
    -What programming language and libraries and framework you are using 

Runtime (Azure App Services)
    -Predefined Container that has your programming language 
    and commonly used library for that language installed 

    =Choose a runtime=
        -.NET 
        -.NET Core 
        -Java 
        -Ruby
        -Node.js 
        -PHP 
        -Python

    (CSPs will usually stop supporting older versions 
    to enforce customers to keep good security practices 
    by having latest patches) 

==Custom containers==
    -to use different runtimes or bundle in a packages or software==Functions==
    -Managed VMs running managed containers 
    -Known as SERVERLESS COMPUTE 
    -Upload piece of code, choose amount of memory and duration
    -You are Responsible for ONLY code and data 
    -Very cost effective (only pay for the time code is running)
        -VMs only run when there is code in it 
        -Cold Starts is a side-effect of this setup 


3.Global infrastructure
==Regions and Geographies==

Region
    -Grouping of multiple datacenters (Availability zones)

    --Azure has 58 regions available across 140 countries--

Geography
    -discreet market of two or more regions that preserves
    DATA RESIDENCY and COMPLIANCE BOUNDARIES

=Azure geographies=
    -United states
    -Azure Government(US)
    -Canada
    -Brazil
    -Mexico
    
==Paired Regions==
-Each Region is paired with another region 300 MILES AWAY
-One region updated at a time(ENSURE NO OUTAGES)
-Some AZURE services rely on Paired regions for DISASTER RECOVERY 

AZURE GEO-REDUNDANT STORAGE (GRS)
    -Replicates data to a secondary region Automatically
    (ensuring data is durable even when primary region isn't recoverable)


==Region Types and Service Availability==
(Not all Cloud services available in every Region)

Recommended Region
    -Region that provides broadest range of service capabilities
    DESIGNED TO SUPPORT AVAILABILITY ZONES now or in the future

Alternate (other) Region 
    -Region that extends Azure's footprints within a data residency boundary
    (Not designed to support AZ's=Availability Zones) 
    They are labeled as OTHER in the Azure Portal

=General Availability (GA)=
    -Service considered ready to be used publicly by everyone

Azure Cloud services are grouped into three categories:
(Category determines when cloud services become available)

    -Foundational
        When GA, immediately or in 12 months (Recommended or Alternate Region)-Premium 
                    -scale to 10 dedicated instances 
                    -availability SLA of 99.95%
                    -multiple levels of hardware


            =Isolated Tab=
                =Isolated Tier 
                    -dedicated Azure virtual Network 
                    -full netword and compute isolation 
                    -scale out to 100 instances 
                    -availability SLA of 99.95%


6. Azure Container Instances

==Introduction to ACI==
    -PACKAGE, MANAGE, DEPLOY cloud applications using CONTAINERS 
    (Fully managed DOCKER AS A SERVICE)

    -Allow you to LAUNCH CONTAINERS without the need to worry about 
    configuring or managing the underlying virtual machine

    -Designed for isolate containers:
        -simple applications 
        -task automation 
        -build jobs 

    =Why CONTAINERS ARE BETTER OPTION THAN VMs=
        -containers provisioned in seconds , VMs in minutes 
        -containers biled per second ,VMs billed per hour (greater savings)
        -containers GRANULAR AND CUSTOM SIZING OF vCPUs, Memory, and GPUs , VMS sizes are predetermined
        -ACI can deploy both Windows and Linux containers
        -PERSIST STORAGE WITH AZURE FILES for your ACI containers 
        -ACIs are accessed via fully qualified domain name (FQDN) -> eg. customlabel.azureregion.azurecontainer.io.
        
        Azure provides Quickstart images to start launching example applications but can also source from:
            -AZURE CONTAINER REGISTRY 
            -DOCKER HUB 
            -PRIVATELY HOSTED CONTAINER REGISTRY 

    =Container groups=
        -collection of containers that get scheduled on the same host machine 

        Group share:
            -lifecycle 
            -resources 
            -local Network
            -storage volumes
        
        CONTAINER GROUPS are similar to KUBERNETES POD 

        Multi-container groups only support Linux containers currently-Update Domain 
        Azure may need to apply udpated to hardware and software 
        Ensure your resources do not go offline 

    -Availability set 
        Logical grouping to ensure that VMs you place in the availabiliy set
        are different fault/update domains to avoid downtime

4. Compute services
==Computing services==

    AZURE VIRTUAL MACHINES 
        -Windows or Linux virtual machines (VMs)
            Choose OS, Memory, CPU, Storage (Share hardware with customers)

    AZURE CONTAINER INSTANCES 
        -DOCKER AS A SERVICE-
        Run containerized apps on Azure without provisioning servers of VMs

    AZURE KUBERNETES SERVICE (AKS)
        -KUBERNETES AS A SERVICE.
            Easy to deploy, manage and scale containerized applications
            (Uses the open source Kubernetes (K8) software) 

    AZURE SERVICE FABRIC 
        -TIER-1 ENTERPRISE CONTAINERS AS A SERVICE
            (Distributed systems platform. Runs in Azure or On-Premises)
            Easy to package, deploy, and manage scalable and reliable MICROSERVICES 

    AZURE FUNCTIONS 
        - Event-driven, serverless compute(functions) run code without 
        provisioning or managing servers.
        You pay only for the compute time you consume 

    AZURE BATCH
        -Plans,schedules and executres your batch computer workloads accross running 100+ jobs in parallel.
        Use SPOT VMs to save money(previously used Low-priority VMs to save on compute)


==AZURE VIRTUAL MACHINES==
    -Highly configurable server.
    Virtualization (run a server WITHOUT HAVING TO BUY AND MAINTAIN PHYSICAL HARDWARE )

    -Virtual machines maintenance 
        Applying OS system patches 
        Installing and configuring packages 

    -Things to know
        -The size of the virtual machine is determined by the IMAGE 
            Image(combination of vCPU's, Memory, and Storage capacity)

In [9]:
# provide the llm to use 
llm = ChatOpenAI(
    model='gpt-oss-120b',
    base_url=os.getenv('CEREBRAS_URL'),
    api_key=os.getenv('CEREBRAS_API_KEY')
)

In [10]:
# provide the llm from mistral 
llm_mistral = ChatOpenAI(
    model='devstral-2512',
    base_url=os.getenv('MISTRAL_URL'),
    api_key=os.getenv('MISTRAL_API_KEY')
)

In [11]:
# see how many tokens in a text (using RAG)
import tiktoken 

# get the encodinga
encoding = tiktoken.get_encoding('cl100k_base')

# encode the text to get the tokens
tokens = encoding.encode(content_answer)

print(len(tokens))

3931


In [13]:
messages = [
        SystemMessage("You are a professional Azure teacher, and you provide good explanations for the user's questions." \
                      "You provide good and understandable explanations to prepare a student for the Azure 900 fundamentals exam" \
                    "You base your answers from this information retrieved:"),
        HumanMessage('hello world')
    ] 

messages

[SystemMessage(content="You are a professional Azure teacher, and you provide good explanations for the user's questions.You provide good and understandable explanations to prepare a student for the Azure 900 fundamentals examYou base your answers from this information retrieved:", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello world', additional_kwargs={}, response_metadata={})]

In [17]:
# provide the function to use in gradio 
def rag_chatbot(user_message:str, history:str): 
    # clear the metadata 
    history = [{"role":h["role"], "content":h["content"]} for h in history]

    # provide the answer from the retriever
    retrieved_info = retriever.invoke(user_message)

    # join the page content info 
    context = "".join([content.page_content for content in retrieved_info])
    
    # provide the history 
    messages = [
        SystemMessage("You are a professional Azure teacher, and you provide good explanations for the user's questions." \
                      "You provide good and understandable explanations to prepare a student for the Azure 900 fundamentals exam" \
                    "You base your answers from this information retrieved: \n{context}".format(context=context)),
        HumanMessage(user_message)
    ] + history


    # you provide the from the llm  
    response = llm.stream(messages) # with typewriter functionalilty

    # provide the empty string to yield the chunks to 
    final_response = ""

    # provide the generator (for each ready to display chunk)
    for chunk in response:
        if chunk.content is not None:
            final_response+=chunk.content
            yield final_response

        else:
            yield final_response

In [33]:
# use gradio 
rag_chat = gr.ChatInterface(
    fn=rag_chatbot,
    chatbot=gr.Chatbot(
        height=1000,
        label="Azure 900 Exam"
    ),
    title='Azure 900 Fundamentals Teacher',
    flagging_mode='never',
    examples=["Which are the Azure services?"],
    show_progress='minimal'
)

In [39]:
# launch the interface 
rag_chat.launch(
    inbrowser=True,
    css='footer {display: none !important}',
    theme=gr.themes.Glass()
)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7860/: Operation not supported


In [41]:
gr.close_all()